<a href="https://colab.research.google.com/github/woghd8503/kernel-academy-web-crawling/blob/main/sele_colab_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
###############################
# 1. 필수 패키지 최신화 및 설치
###############################
sudo apt -y update
sudo apt install -y wget curl unzip jq

###############################
# 2. libu2f-udev (필요 시)
###############################
wget -q http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
sudo dpkg -i libu2f-udev_1.1.4-1_all.deb

###############################
# 3. Google Chrome 설치
###############################
wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
sudo dpkg -i google-chrome-stable_current_amd64.deb || sudo apt -fy install

###############################
# 4. 설치된 Chrome 버전 확인
###############################
CHROME_VERSION=$(google-chrome --version | awk '{print $3}')
echo "Chrome version: $CHROME_VERSION"

###############################
# 5. Chrome 115+ 버전용 ChromeDriver 다운로드
###############################
# Chrome 115부터는 Chrome for Testing에서 다운로드
CHROME_MAJOR_VERSION=$(echo $CHROME_VERSION | cut -d. -f1)

if [ $CHROME_MAJOR_VERSION -ge 115 ]; then
    # 새로운 Chrome for Testing 엔드포인트 사용
    echo "Chrome 115+ detected, using new download method..."

    # 사용 가능한 버전 목록 가져오기
    VERSIONS_JSON=$(curl -s https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)

    # Chrome 버전과 가장 일치하는 ChromeDriver 버전 찾기
    CHROMEDRIVER_URL=$(echo $VERSIONS_JSON | jq -r --arg chrome_version "$CHROME_VERSION" '
        .versions |
        map(select(.version | startswith($chrome_version | split(".")[0:2] | join(".")))) |
        last |
        .downloads.chromedriver[] |
        select(.platform == "linux64") |
        .url
    ')

    # URL이 없으면 최신 stable 버전 사용
    if [ -z "$CHROMEDRIVER_URL" ]; then
        echo "Exact version not found, using latest stable..."
        CHROMEDRIVER_URL=$(curl -s https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json | \
            jq -r '.channels.Stable.downloads.chromedriver[] | select(.platform == "linux64") | .url')
    fi

    echo "Downloading ChromeDriver from: $CHROMEDRIVER_URL"
    wget -q "$CHROMEDRIVER_URL" -O /tmp/chromedriver-linux64.zip

    # 압축 해제 (새로운 구조는 chromedriver-linux64 폴더 안에 있음)
    unzip -q /tmp/chromedriver-linux64.zip -d /tmp/
    sudo mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
    sudo chmod +x /usr/local/bin/chromedriver
else
    # Chrome 114 이하 버전은 기존 방식 사용
    echo "Chrome 114 or older detected, using legacy download method..."
    CHROMEDRIVER_VERSION=$(curl -sS "https://chromedriver.storage.googleapis.com/LATEST_RELEASE_${CHROME_MAJOR_VERSION}")
    wget -q -N "https://chromedriver.storage.googleapis.com/${CHROMEDRIVER_VERSION}/chromedriver_linux64.zip" -P /tmp/
    unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
    chmod +x /tmp/chromedriver
    sudo mv /tmp/chromedriver /usr/local/bin/chromedriver
fi

###############################
# 6. 설치 확인
###############################
echo "Chrome version: $(google-chrome --version)"
echo "ChromeDriver version: $(chromedriver --version)"

###############################
# 7. Selenium 파이썬 패키지 설치
###############################
pip install -q selenium

echo "Installation completed!"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,741 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,553 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,295 kB]
Hit:13 https://ppa.lau

#접속 테스트

In [ ]:
from selenium import webdriver

URL = "https://google.com"

options = webdriver.ChromeOptions()
options.add_argument("--headless") # 로컬 기준 헤드리스를 안주고 실행하는 경우 : 개발할때
# 로컬에서 헤드리스를 주는 경우 (백그라운드) : 운영 할 때  -> 빠르고 GUI 자원 안먹고, 디버깅은 어려움
options.add_argument('--disable-dev-shm-usage')
# 공유 메모리 사용 비활성화 도커같은 가상환경 상에서 안정성 향상, 메모리 부족으로 크래쉬 방지
options.add_argument("--no-sandbox")
# 샌드박스 브라우저 프로세스를 격리 , 시스템 보안을 강화
# 노 샌드 박스 -> 보안 크롤링을 더 자유롭게 할 수 있게함 -> 양날의칼


driver = webdriver.Chrome(
    options=options
)

driver.get(URL)
title = driver.title
driver.close()

print(title)

Google


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By # css, tag 선택자
from selenium.webdriver.common.keys import Keys # 선택자가 선택을 한 부분에 대해 컨트롤러 (엔터, 탭, ESC, 스페이스바 )
from selenium.webdriver.support.ui import WebDriverWait # 조건이 만족할 때 까지 지능적으로 '대기'
'''
time sleep 을 거는 경우 (복습)
1.웹 마다 늦게 뜨는 경우
2. 봇이 아닌척
'''

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
# '대기' 조건 설정 가능
# HTML 페이지 안에 , 슬라이더가 보일 때, 잠깐 기다려야 점진적으로 보이는 경우, 팝업이 접속 이후 보이는 경우

import pandas as pd
from datetime import datetime
import sys

#  드라이버 설정 및 페이지 접속

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(options=chrome_options)



#검색 키워드 설정

In [ ]:

keyword = "인공지능"
search_url = f"https://news.google.com/search?q={keyword}&hl=ko&gl=KR&ceid=KR%3Ako"

print(f"🔍 '{keyword}' 검색 페이지 접속 중...")
driver.get(search_url)
time.sleep(3)
print("✅ 페이지 로딩 완료")

🔍 '인공지능' 검색 페이지 접속 중...
✅ 페이지 로딩 완료


In [ ]:
articles = driver.find_elements(By.TAG_NAME,"article")
print(f"📰 찾은 article 태그 수: {len(articles)}")


📰 찾은 article 태그 수: 109


In [ ]:

# 첫 번째 article 저장
if articles:
    first_article = articles[0]
    print("✅ 첫 번째 article 선택 완료")
else:
    print("❌ article 태그를 찾을 수 없습니다.")


✅ 첫 번째 article 선택 완료


In [ ]:
if articles:
    # 모든 태그 확인
    # 정규 표현식 방식으로 태그를 찾겠다
    all_tags = first_article.find_elements(By.XPATH, './/*')
    # . = 현재 , // 현재부터 모든 하위 노드 , * 모든것, 와일드 카드
    '''
    <article>
        1레벨 <p> ~~
        1레벨 <p>
        1레벨<div>
            2레벨<a>
        1레벨</div>
    '''
    # ./* = > 1레벨 만 가져온다 or 직접 적인 자식만 가져온다
    # .//div => 모든 div태그 불러오기
    '''
    특정 태그 제외 하고 불러 오는 방법 (xpath)
    //*[not(self::script)]
    //*[not(self::img)]
    //*[not(self::a)]

    선택자로 제외(By.CSS_SELECTOR + find elements)
    find_elements(By.CSS_SELECTOR, '*:not(img)')
    '''

    tag_names = set([tag.tag_name for tag in all_tags[:30]])
    print(f"📌 article 내부 태그 종류: {tag_names}")

    # 태그별 개수 확인
    print(f"\n📊 주요 태그 개수:")
    print(f"   - a 태그: {len(first_article.find_elements(By.TAG_NAME, 'a'))}")
    print(f"   - div 태그: {len(first_article.find_elements(By.TAG_NAME, 'div'))}")
    print(f"   - span 태그: {len(first_article.find_elements(By.TAG_NAME, 'span'))}")
    print(f"   - time 태그: {len(first_article.find_elements(By.TAG_NAME, 'time'))}")

📌 article 내부 태그 종류: {'time', 'path', 'span', 'img', 'a', 'button', 'figure', 'div', 'svg'}

📊 주요 태그 개수:
   - a 태그: 2
   - div 태그: 17
   - span 태그: 2
   - time 태그: 1


In [ ]:
if articles:
    links = first_article.find_elements(By.TAG_NAME,"a")
    print(f"\n🔗 첫 번째 article의 a 태그 {len(links)}개 상세 정보:")


🔗 첫 번째 article의 a 태그 2개 상세 정보:


In [ ]:
for i, link in enumerate(links):
    href = link.get_attribute('href') or 'No href'
    text = link.text.strip() or 'No text'
    print(f"\n링크 {i+1}:")
    print(f"  - 텍스트: {text}...")
    print(f"  - href: {href}...")


링크 1:
  - 텍스트: No text...
  - href: https://news.google.com/read/CBMickFVX3lxTE9EUGhoNWUyMURDRU8zY2tWYjRyZUItc1FCeUstYXhrMXZvRXF0aFpXR2JiREtWNGFLcVRuR2o0dDFZWDJVczRjWmNDZzlCb2FMd1R2WVR2X3JUdm00OHFsZF9vc2lQUzRXSFVOemY1Q1VDd9IBdkFVX3lxTE53YW5rNWc4VmVKbm1JZHNBZHowLS01T0wzNmJvcEVEMVd0N3A1ZXNRV1g1NkxtV0hKWFZ3ZFRFNmwwZ3hBcXlRRTdDOFZSamJGczZpMUo5ZnBjOUJxNnpIT3pId2UzNUVYQ3pROFJ3ZWNCbTJJVFE?hl=ko&gl=KR&ceid=KR%3Ako...

링크 2:
  - 텍스트: 이재명 정부 ‘AI정책수석’ 신설···임문영 유력...
  - href: https://news.google.com/read/CBMickFVX3lxTE9EUGhoNWUyMURDRU8zY2tWYjRyZUItc1FCeUstYXhrMXZvRXF0aFpXR2JiREtWNGFLcVRuR2o0dDFZWDJVczRjWmNDZzlCb2FMd1R2WVR2X3JUdm00OHFsZF9vc2lQUzRXSFVOemY1Q1VDd9IBdkFVX3lxTE53YW5rNWc4VmVKbm1JZHNBZHowLS01T0wzNmJvcEVEMVd0N3A1ZXNRV1g1NkxtV0hKWFZ3ZFRFNmwwZ3hBcXlRRTdDOFZSamJGczZpMUo5ZnBjOUJxNnpIT3pId2UzNUVYQ3pROFJ3ZWNCbTJJVFE?hl=ko&gl=KR&ceid=KR%3Ako...


In [ ]:
if articles: #xpath
    text_elements = first_article.find_elements(By.XPATH, './/*[text()]') # text 태그를 찾는게 아닌, A태그 안에, text(value)을 가져오는 것
    print(f"\n📝 텍스트가 있는 요소 수: {len(text_elements)}")
    print(text_elements[2].text)




📝 텍스트가 있는 요소 수: 4
이재명 정부 ‘AI정책수석’ 신설···임문영 유력


In [ ]:
print("\n주요 텍스트 내용:")
for i, elem in enumerate(text_elements[:15]):
    text = elem.text.strip()
    print(text)


주요 텍스트 내용:
시사저널e

이재명 정부 ‘AI정책수석’ 신설···임문영 유력
어제


In [ ]:
driver.quit()
print("\n✅ 구조 확인 완료. 드라이버 종료됨.")


✅ 구조 확인 완료. 드라이버 종료됨.


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
keyword = "삼성전자"
max_count = 30
search_url =  f"https://news.google.com/search?q={keyword}&hl=ko&gl=KR&ceid=KR%3Ako"

print(f"🔍 '{keyword}' 크롤링 시작...")
driver.get(search_url)
time.sleep(3)

🔍 '삼성전자' 크롤링 시작...


In [ ]:
articles = driver.find_elements(By.TAG_NAME, "article")
print(f"📰 {len(articles)}개의 article 발견")

# 결과 저장 리스트
results = []

📰 108개의 article 발견


In [ ]:
if articles:
    article = articles[1]
    article_data = {
        'no': 1,
        'title': '',
        'source': '',
        'time': '',
        'link': '',
        'keyword': keyword
    }
    # a 태그에서 제목과 링크 찾기
    a_tags = article.find_elements(By.TAG_NAME,'a')

    for a in a_tags:
        href = a.get_attribute("href") or ""
        text = a.text.strip()

        # 가장 긴 텍스트를 제목으로
        if text and len(text) > len(article_data['title']):
            article_data['title'] = text
            if '/articles/' in href or 'http' in href:
                article_data['link'] = href
            print('제목 :' , text)

    print(f"\n📌 선택된 제목: {article_data['title'][:50]}...")
    print(f"📌 링크: {article_data['link']}...")


제목 : 대만 시장조사업체 "삼성전자, 5나노 기반 자율주행차용 반도체 개발 나서"

📌 선택된 제목: 대만 시장조사업체 "삼성전자, 5나노 기반 자율주행차용 반도체 개발 나서"...
📌 링크: https://news.google.com/read/CBMic0FVX3lxTFBKQmRXbHBfd2wybmVZam5NenlmWVRqMzFyV3NFb1o4VmxST016U3hNTjFmampXNklfdHF5MHVPMnd4dFdPMzdERWxTdFVVcFFNQ2M1R1pERnFNLVM1VXhUQl9rdG5lQUR3YUhHUDJGV3kya3c?hl=ko&gl=KR&ceid=KR%3Ako...


In [ ]:
time_elements = article.find_elements(By.TAG_NAME, 'time')
time_elements[0].text
time_elements[0].get_attribute('datetime')

'2025-06-05T01:26:56Z'

In [ ]:
if articles and article_data['title']:
    # 시간 정보
    time_elements = article.find_elements(By.TAG_NAME, 'time')
    if time_elements:
        article_data['time'] = time_elements[0].text or time_elements[0].get_attribute('datetime')
        print(f"📌 시간: {article_data['time']}")

    # 언론사 찾기
    all_texts = article.find_elements(By.XPATH, './/*[text()]')
    for elem in all_texts:
        text = elem.text.strip()
        # 짧은 텍스트 중에서 언론사 찾기
        if text and 2<= len(text) <= 20 and not any(c.isdigit() for c in text):
            # 일단, 텍스트가 존재 하는지 확인
            # 텍스트의 길이가 2글자 이상이고(어제, 엊그제)
            # 텍스트를 한글자씩 파싱을 진행하면서, 글자가 숫자(isdigit)인지 판단하고 한글자도 숫자가 아니여야지만
            # True 리턴(any)
            if text not in article_data['title']:
                article_data['source'] = text
                print(f'언론사 : {text}')
                break
    # 결과 추가
    if article_data['title']:
        results.append(article_data)
        print(f"\n✅ 첫 번째 기사 수집 완료!")

📌 시간: 1시간 전
언론사 : 비즈니스포스트

✅ 첫 번째 기사 수집 완료!


In [ ]:
print(f"\n🔄 나머지 기사 수집 중...")

for i in range(1, min(max_count, len(articles))):
    try:
        article = articles[i]
        article_data = {
            'no': i + 1,
            'title': '',
            'source': '',
            'time': '',
            'link': '',
            'keyword': keyword
        }

        # a 태그에서 제목과 링크 찾기
        a_tags = article.find_elements(By.TAG_NAME,'a')
        for a in a_tags:
            href = a.get_attribute('href') or ''
            text = a.text.strip()

            if text and len(text) > len(article_data['title']):
                article_data['title'] = text
                if '/article/' in href or 'http' in href:
                    article_data['link'] = href

        # 시간 정보
        time_elements = article.find_elements(By.TAG_NAME, 'time')
        if time_elements:
            article_data['time'] = time_elements[0].text or time_elements[0].get_attribute("datetime")

        # 언론사
        all_texts = article.find_elements(By.XPATH, './/*[text()]')
        for elem in all_texts:
            text = elem.text.strip()
            if text and 2<= len(text) <= 20 and not any(c.isdigit() for c in text):
                if text not in article_data['title']:
                    article_data['source'] = text
                    print(f'언론사 : {text}')
                    break

        # 유효한 결과만 추가
        if article_data['title']:
            results.append(article_data)
            print(f"✅ {i+1}/{max_count} 수집 완료: {article_data['title'][:30]}...")

    except Exception as e:
        print(f"⚠️ {i+1}번째 기사 처리 중 오류: {str(e)[:50]}")
        continue


🔄 나머지 기사 수집 중...
언론사 : 비즈니스포스트
✅ 2/30 수집 완료: 대만 시장조사업체 "삼성전자, 5나노 기반 자율주행차용...
언론사 : 한국경제
✅ 3/30 수집 완료: 'TSMC 출신 인재 모셔왔다'…삼성전자 '올인'...
언론사 : YTN 사이언스
✅ 4/30 수집 완료: 삼성전자, '울트라급' 갤럭시Z 폴드7 7월 출시 예고...
언론사 : 연합뉴스
✅ 5/30 수집 완료: 삼성, '울트라급' 폴더블 신작 '갤럭시 Z 폴드7' ...
언론사 : 작성자: 박지은
✅ 6/30 수집 완료: 삼성전자, 갤Z폴드7 예고 "울트라급 경험 제시할 것"...
언론사 : 연합뉴스
✅ 7/30 수집 완료: 삼성전자, 17∼19일 글로벌 전략회의…하반기 사업 전...
언론사 : 글로벌이코노믹
✅ 8/30 수집 완료: 삼성전자, 17일 글로벌 전략회의 개최…하반기 사업전략...
언론사 : 지디넷코리아
✅ 9/30 수집 완료: 삼성전자, 17~19일 글로벌 전략회의…불확실성 돌파구...
언론사 : 경향신문
✅ 10/30 수집 완료: SK하이닉스, ‘33년 신화’ 삼성전자 꺾었다...
언론사 : 연합뉴스
✅ 11/30 수집 완료: '33년 신화' 삼성전자 꺾은 SK하이닉스…다음 격전지...
언론사 : 비즈니스포스트
✅ 12/30 수집 완료: JP모간 "미국 EDA 중국 규제로 CXMT DDR5 ...
언론사 : 네이트 뉴스
✅ 13/30 수집 완료: 삼성전자, '중남미 테크 세미나'…'비스포크 AI 가전...
언론사 : 비즈니스포스트
✅ 14/30 수집 완료: 삼성전자 갤럭시 AI 이미지 편집 '포토 어시스트' 사...
언론사 : lawissue.co.kr
✅ 15/30 수집 완료: 삼성전자 “갤럭시 AI '포토 어시스트' 최근 3개월간...
언론사 : 부산일보
✅ 16/30 수집 완료: 삼성전자, 갤럭시 AI ‘포토 어시스트’ 사용률 배 증...
언론사 : 인사이트코리아
✅ 17/30 수집 완료: 이재용 삼성전자 회장 “국적·성별 불문...해외 특급 ...
언

In [ ]:
driver.quit()

print(f"\n📊 총 {len(results)}개 기사 수집 완료!")
print("="*80)

for article in results:
    print(f"\n{article['no']}. {article['title']}")
    print(f"   언론사: {article['source']}")
    print(f"   시간: {article['time']}")
    print(f"   링크: {article['link'][:50]}..." if article['link'] else "   링크: 없음")



📊 총 30개 기사 수집 완료!

1. 대만 시장조사업체 "삼성전자, 5나노 기반 자율주행차용 반도체 개발 나서"
   언론사: 비즈니스포스트
   시간: 1시간 전
   링크: https://news.google.com/read/CBMic0FVX3lxTFBKQmRXb...

2. 대만 시장조사업체 "삼성전자, 5나노 기반 자율주행차용 반도체 개발 나서"
   언론사: 비즈니스포스트
   시간: 1시간 전
   링크: https://news.google.com/read/CBMic0FVX3lxTFBKQmRXb...

3. 'TSMC 출신 인재 모셔왔다'…삼성전자 '올인'
   언론사: 한국경제
   시간: 3일 전
   링크: https://news.google.com/read/CBMiWkFVX3lxTFBybmlKQ...

4. 삼성전자, '울트라급' 갤럭시Z 폴드7 7월 출시 예고
   언론사: YTN 사이언스
   시간: 3시간 전
   링크: https://news.google.com/read/CBMijAFBVV95cUxPVmdOZ...

5. 삼성, '울트라급' 폴더블 신작 '갤럭시 Z 폴드7' 내달 출시 예고
   언론사: 연합뉴스
   시간: 어제
   링크: https://news.google.com/read/CBMiW0FVX3lxTE1tSWJOR...

6. 삼성전자, 갤Z폴드7 예고 "울트라급 경험 제시할 것"
   언론사: 작성자: 박지은
   시간: 21시간 전
   링크: https://news.google.com/read/CBMiTEFVX3lxTE5PY0xXS...

7. 삼성전자, 17∼19일 글로벌 전략회의…하반기 사업 전략 논의
   언론사: 연합뉴스
   시간: 2시간 전
   링크: https://news.google.com/read/CBMiW0FVX3lxTE94RnR2V...

8. 삼성전자, 17일 글로벌 전략회의 개최…하반기 사업전략 마련
   언론사: 글로벌이코노믹
   시간: 2시간 전
   링크: https://new

In [ ]:
df = pd.DataFrame(results)
print(f"\n✅ DataFrame 생성 완료 (총 {len(df)}개 기사)")
df



#RSS
## requests -> RSS -> selenium

### rss vs selenium
### 속도 rss 훨씬 빠르다
### 안정성 rss  win!
### 데이터 품질 rss win
### rss를 제대로 지원 안하는 경우, 확장성은 selenium
### 봇 탐지 위험은 rss는 서버내에서 지원해주는 기능이라 rss는 자유롭게 사용 가능
### 리소스 rss win


In [ ]:
!pip install feedparser

import feedparser


In [ ]:

def get_google_news_rss(keyword):
    """구글 뉴스 RSS 피드 사용"""
    rss_url = f"https://news.google.com/rss/search?q={keyword}&hl=ko&gl=KR&ceid=KR:ko"

    print(f"🔍 RSS 피드로 '{keyword}' 검색 중...")
    feed = feedparser.parse(rss_url)

    results = []
    for i, entry in enumerate(feed.entries):
        results.append({
            'no': i + 1,
            'title': entry.title,
            'link': entry.link,
            'time': entry.get('published', ''),
            'source': entry.get('source', {}).get('title', 'Unknown'),
            'keyword': keyword
        })
        print(f"✅ {i+1}. {entry.title[:50]}...")

    return results

# RSS 방법으로 뉴스 가져오기
rss_articles = get_google_news_rss("머신러닝")

# DataFrame으로 변환
df_rss = pd.DataFrame(rss_articles)
print(f"\n📊 RSS로 {len(df_rss)}개 기사 수집 완료")
df_rss


🔍 RSS 피드로 '머신러닝' 검색 중...
✅ 1. 애드포러스, AI 머신러닝 기반 시나리오 최적화 시장 진출 - 뉴시스...
✅ 2. "양자 컴퓨팅, 머신러닝의 한계를 허물다"... 광자 프로세서로 이진 분류 혁신 - 인공지...
✅ 3. 알테어, ‘가트너 매직 쿼드런트’ 데이터 과학·머신러닝 플랫폼 리더 선정 - 캐드앤그래픽스...
✅ 4. 알테어, ‘가트너 매직 쿼드런트’ 데이터 과학·머신러닝 플랫폼 부문 2년 연속 리더 선정 ...
✅ 5. 알테어, ‘데이터 과학·머신러닝’ 플랫폼 선도 - 데이터넷...
✅ 6. 알테어, '가트너 매직 쿼드런트' 데이터 과학·머신러닝 플랫폼 부문 2년 연속 리더 선정 ...
✅ 7. 10년 2025월 최고의 머신러닝 및 AI 뉴스레터 XNUMX선 - Unite.AI...
✅ 8. AI ·머신러닝 갖춘 암젠, 신약개발 능력 무서울 만큼 빠르고 정확해졌다 - 팜뉴스...
✅ 9. 콜로퀴움 "LLM을 포함한 머신러닝 모델의 학습과 추론을 위한 스케줄링 최적화" 서지원 교...
✅ 10. CFD와 머신러닝을 활용한 공력 성능 예측 프로세스 개발 - 캐드앤그래픽스...
✅ 11. 경희대병원, 머신러닝 기반 제2형 당뇨병 5년 내 발병 예측 모델 개발 - 메디팜스투데이...
✅ 12. 데이터와 머신러닝을 통해 새로운 금융 가치를 발굴하는, 토스 Credit Modeling ...
✅ 13. 양자머신러닝, 의료 등 산업미래 바꾼다 - 브릿지경제...
✅ 14. 마우저, 로보틱스·AI·머신러닝 집중 조명 - e4ds news...
✅ 15. "양자 인공지능 대중화 이끈다!"...英 STFC 하트리센터-IBM, 오픈소스 양자 머신러...
✅ 16. 10년 2025월 최고의 머신러닝 소프트웨어 XNUMX선 - Unite.AI...
✅ 17. 머신러닝에 가장 적합한 언어는 무엇일까요? (2025년 XNUMX월) - Unite.AI...
✅ 18. “아!, 머신러닝이 이런 거구나"...박민서 교수, 입문에서 활용까지의 지침서 '머신러닝 .

,no,title,link,time,source,keyword
0,1,"애드포러스, AI 머신러닝 기반 시나리오 최적화 시장 진출 - 뉴시스",https://news.google.com/rss/articles/CBMiYEFVX...,"Thu, 05 Jun 2025 00:44:44 GMT",뉴시스,머신러닝
1,2,"""양자 컴퓨팅, 머신러닝의 한계를 허물다""... 광자 프로세서로 이진 분류 혁신 -...",https://news.google.com/rss/articles/CBMiZ0FVX...,"Tue, 03 Jun 2025 02:32:57 GMT",인공지능신문,머신러닝
2,3,"알테어, ‘가트너 매직 쿼드런트’ 데이터 과학·머신러닝 플랫폼 리더 선정 - 캐드앤...",https://news.google.com/rss/articles/CBMijgFBV...,"Wed, 04 Jun 2025 05:24:11 GMT",캐드앤그래픽스,머신러닝
3,4,"알테어, ‘가트너 매직 쿼드런트’ 데이터 과학·머신러닝 플랫폼 부문 2년 연속 리더...",https://news.google.com/rss/articles/CBMiaEFVX...,"Wed, 04 Jun 2025 02:22:19 GMT",테크월드뉴스,머신러닝
4,5,"알테어, ‘데이터 과학·머신러닝’ 플랫폼 선도 - 데이터넷",https://news.google.com/rss/articles/CBMibEFVX...,"Wed, 04 Jun 2025 07:37:28 GMT",데이터넷,머신러닝
...,...,...,...,...,...,...
95,96,"""의료기기 개발 쉽도록""…'머신러닝 국제공통 지침' 발간 - 네이트 뉴스",https://news.google.com/rss/articles/CBMiU0FVX...,"Wed, 26 Mar 2025 07:00:00 GMT",네이트 뉴스,머신러닝
96,97,"무신사 머신러닝 기업 몰로코와 업무협약, 인공지능 바탕 고객 경험 개선 - 비즈니스포스트",https://news.google.com/rss/articles/CBMic0FVX...,"Fri, 31 Jan 2025 08:00:00 GMT",비즈니스포스트,머신러닝
97,98,"NIA, '2024 양자정보기술 백서' 발간...양자AI, 양자머신러닝 동향 및 전...",https://news.google.com/rss/articles/CBMiZ0FVX...,"Mon, 30 Dec 2024 08:00:00 GMT",인공지능신문,머신러닝
98,99,"서울대 공대 전기정보공학부 곽정훈 교수 연구팀, 머신러닝을 이용한 유기열전소자 성능...",https://news.google.com/rss/articles/CBMiX0FVX...,"Mon, 02 Dec 2024 08:00:00 GMT",뉴스와이어,머신러닝


In [ ]:
# html 문서를 parsing -> bsoup 으로 파싱
# html 문서를 가져오는것 -> requests , rss
# html 문서에 이벤트가 있는 특수한 경우 -> selenium (파싱)

## 추후 과제(기사를 클릭해서 본문 내용 긁어오기)
# 자주쓰이는 선택자 리스트 뽑아 놓기
content_selectors = [
    "article",
    "[role='article']",
    ".article-body",
    ".article-content",
    ".content",
    "#article-body",
    "#content",
    ".story-body",
    ".entry-content",
    "main article",
    "[itemprop='articleBody']",
    ".post-content",
    ".news-content"
]
# css_selector로 해당 태그를 찾고 텍스트를 다 찾기
# 텍스트 중 가장 긴 텍스트 => 본문
# 본문을 못찾은 경우 p태그를 전체 수집해서 하나의 텍스트 변수로 저장하기